In [ ]:
!pip install -q transformers

In [ ]:
from transformers import pipeline

In [ ]:
model = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [ ]:
ouput = model(ARTICLE, max_length=130, min_length=30, do_sample=False)

In [ ]:
display(ouput[0]['summary_text'])

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

In [ ]:
ds

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
dataset = pd.read_csv("train.csv")

In [ ]:
dataset.head(2)

In [ ]:
dataset.shape

In [ ]:
display(dataset[['dialogue', 'summary']].iloc[0:1])

In [ ]:
dialogue_1 = dataset['dialogue'].iloc[0:1].values[0]
summary_1 = dataset['summary'].iloc[0:1].values[0]

In [ ]:
ouput = model(dialogue_1, max_length=130, min_length=30, do_sample=False)
display(ouput[0]['summary_text'])

In [ ]:
display(summary_1)

# Fine Tuning

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [ ]:
print(tokenizer)

In [ ]:
print(model)

In [ ]:
def text_preprocessing(ds):
  source = ds['dialogue']
  target = ds['summary']

  source_tokenized = tokenizer(source, padding='max_length', truncation=True, max_length=512)
  target_tokenized = tokenizer(target, padding='max_length', truncation=True, max_length=128)

  labels = target_tokenized['input_ids']
  labels = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels]

  model_inputs = {
      'input_ids': source_tokenized['input_ids'],
      'attention_mask': source_tokenized['attention_mask'],
      'labels': labels
  }

  return model_inputs

In [ ]:
from datasets import Dataset
dataset_hf = Dataset.from_pandas(dataset)

In [ ]:
dataset_hf

In [ ]:
df_source = dataset_hf.map(text_preprocessing, batched=True, batch_size=1, remove_columns=['id', 'topic'])

In [ ]:
df_source

In [ ]:
df_source[0]['labels']

In [ ]:
dataset_t = pd.read_csv("train_new.csv")
dataset_hft = Dataset.from_pandas(dataset_t)

In [ ]:
dataset_tst = pd.read_csv("test_new.csv")
dataset_hfts = Dataset.from_pandas(dataset_tst)

In [ ]:
df_source_train = dataset_hft.map(text_preprocessing, batched=True, batch_size=1, remove_columns=['id', 'topic'])
df_source_test = dataset_hfts.map(text_preprocessing, batched=True, batch_size=1, remove_columns=['id', 'topic'])

In [ ]:
df_source_test

# Fine Tuning

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    report_to='wandb'
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=df_source_train,       # training dataset
    eval_dataset=df_source_test          # evaluation dataset
)

In [ ]:
trainer.train()

In [ ]:
eval_result = trainer.evaluate()
eval_result

# Save the model

In [ ]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_tokenizer")

In [ ]:
def new_summarizer(input_text):
  input = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)

  summary = model.generate(input['input_ids'], num_beams=4, max_length=128, min_length=50, early_stopping=True)

  output = tokenizer.decode(summary[0], skip_special_tokens=True)

  return output

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/saved_tokenizer")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/saved_model")

In [ ]:
new_summarizer(ARTICLE)